# [hotsline](https://github.com/AntoineGrimaldi/hotsline) algorithm to replicate results from [this paper](https://www.techrxiv.org/articles/preprint/A_robust_event-driven_approach_to_always-on_object_recognition/18003077/1)
## Load events of the N-MNIST dataset with [Tonic](https://tonic.readthedocs.io/en/latest/index.html)

In [2]:
import tonic
from utils import get_loader

print(f' Tonic version installed -> {tonic.__version__}')

transform = tonic.transforms.NumpyAsType(int)
dataset = tonic.datasets.NMNIST(save_to='../../Data/', train=True, transform=transform)
loader = get_loader(dataset, kfold = 300, shuffle=True)
#get_dataset_info(dataset, properties = ['time', 'mean_isi', 'nb_events']);
print(f'number of samples in the dataset: {len(loader)}')

 Tonic version installed -> 1.0.15
number of samples in the dataset: 200


In [8]:
%cd ../hots
from layer import hotslayer

/home/antoine/homhots/hotsline/hots


In [ ]:
R = (2, 4, 8)
n_neurons = (4, 8, 16)
n_pola = 2
ts_size1 = (2*R[0]+1)**2*n_pola
tau = (5e3, 5e4, 5e5)#7e2
ts_size2 = (2*R[1]+1)**2*n_neurons[0]
ts_size3 = (2*R[2]+1)**2*n_neurons[1]
verbose = True

p_index = dataset.ordering.index('p')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if verbose: print(f'device -> {device}')

torch.set_default_tensor_type("torch.DoubleTensor")

with torch.no_grad():
    output_pola = []
    layer1 = hotslayer(ts_size1, n_neurons[0], device=device)
    #layer1.learning_flag = False
    #layer1.to(device)
    layer2 = hotslayer(ts_size2, n_neurons[1], device=device)
    #layer2.learning_flag = False
    #layer2.to(device)
    layer3 = hotslayer(ts_size3, n_neurons[2], device=device)
    #layer3.learning_flag = False
    #layer3.to(device)
    for events, target in tqdm(loader):
        all_ts, ind_filtered = timesurface(events.squeeze(0), dataset.sensor_size, dataset.ordering, tau = tau[0], surface_dimensions=[2*R[0]+1,2*R[0]+1], filtering_threshold = 2*R[0], device=device)
        layer1.to(device)
        n_star = layer1(all_ts.to(device).squeeze(0))
        layer1.to('cpu')
        if ind_filtered is not None:
            events = events[:,ind_filtered,:]
        events[0,:,p_index] = n_star.cpu()
        sensor_size = [dataset.sensor_size[0], dataset.sensor_size[1], n_neurons[0]]
        del all_ts
        torch.cuda.empty_cache()
        all_ts, ind_filtered = timesurface(events.squeeze(0), sensor_size, dataset.ordering, tau = tau[1], surface_dimensions=[2*R[1]+1,2*R[1]+1], filtering_threshold = 2*R[1], device=device)
        layer2.to(device)
        n_star = layer2(all_ts.to(device).squeeze(0))
        layer2.to('cpu')
        sensor_size = [dataset.sensor_size[0], dataset.sensor_size[1], n_neurons[1]]
        if ind_filtered is not None:
            events = events[:,ind_filtered,:]
        events[0,:,p_index] = n_star.cpu()
        del all_ts
        torch.cuda.empty_cache()
        all_ts, ind_filtered = timesurface(events.squeeze(0), sensor_size, dataset.ordering, tau = tau[2], surface_dimensions=[2*R[2]+1,2*R[2]+1], filtering_threshold = 2*R[2], device=device)
        layer3.to(device)
        n_star = layer3(all_ts.to(device).squeeze(0))
        layer3.to('cpu')
        if ind_filtered is not None:
            events = events[:,ind_filtered,:]
        events[0,:,p_index] = n_star.cpu()
        output_pola.append(n_star.cpu())
        del all_ts
        torch.cuda.empty_cache()